In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/Users/jaemin/Documents/dacon/funda_train.csv')
# 엑셀파일 읽을 때: pd.read_excel

In [3]:
train.head(5)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [4]:
# 잘못된 파일을 불러왔을 수도 있으므로 길이부터 확인한다.
len(train)

6556613

In [5]:
train.describe()

,store_id,card_id,installment_term,amount
count,6.556613e+06,6.556613e+06,6.556613e+06,6.556613e+06
mean,1.084930e+03,2.268127e+06,1.382017e-01,1.043511e+04
std,6.152183e+02,1.351058e+06,1.188152e+00,3.104031e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.771429e+06
25%,5.860000e+02,1.088828e+06,0.000000e+00,2.142857e+03
50%,1.074000e+03,2.239304e+06,0.000000e+00,4.285714e+03
75%,1.615000e+03,3.438488e+06,0.000000e+00,8.571429e+03
max,2.136000e+03,4.663856e+06,9.300000e+01,5.571429e+06


In [6]:
train.columns

Index(['store_id', 'card_id', 'card_company', 'transacted_date',
       'transacted_time', 'installment_term', 'region', 'type_of_business',
       'amount'],
      dtype='object')

In [7]:
train.dtypes

store_id              int64
card_id               int64
card_company         object
transacted_date      object
transacted_time      object
installment_term      int64
region               object
type_of_business     object
amount              float64
dtype: object

### to do: 2019년 3월 1일부터 2019년 5월 31일까지 상점별로 총 매출을 계산
### solution: 2018년 3월 1일부터 2018년 5월 31일까지 상점별 매출액과 같은 값으로 함 (seasonality) 

In [8]:
train_2018 = train[(train['transacted_date']>='2018-03-01') & (train['transacted_date']<='2018-05-31')] 
train_2018.head(5)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
2938,0,855,a,2018-03-01,10:21,0,NaN,기타 미용업,1857.142857
2939,0,1278,g,2018-03-01,14:47,0,NaN,기타 미용업,7142.857143
2940,0,177,h,2018-03-01,16:50,0,NaN,기타 미용업,2000.000000
2941,0,857,a,2018-03-01,16:51,0,NaN,기타 미용업,10000.000000
2942,0,1279,g,2018-03-01,18:29,0,NaN,기타 미용업,7714.285714


In [66]:
train_2018_amount = train_2018.groupby('store_id')['amount'].sum()
train_2018_amount.head(3)

store_id
0    2.171000e+06
1    2.977143e+05
2    9.461429e+05
Name: amount, dtype: float64

In [14]:
train_2018_2 = train_2018.groupby(['store_id']).agg({'installment_term':'mean', 'amount':'sum'})  
# 2개 이상의 컬럼에 각각 다른 함수 적용할 때 : agg({'컬럼명1':'함수명1'}, {'컬럼명2':'함수명2'}, ...)
train_2018_2.head(3)

,installment_term,amount
store_id,,
0,0.105793,2.171000e+06
1,0.000000,2.977143e+05
2,0.219178,9.461429e+05


In [15]:
train_2018['store_id'].nunique()

1955

In [67]:
train_2018_amount.to_csv('/Users/jaemin/Documents/dacon/0812_1.csv', header=True)

In [68]:
read_test = pd.read_csv('/Users/jaemin/Documents/dacon/0812_1.csv')
read_test.head()

,store_id,amount
0,0,2.171000e+06
1,1,2.977143e+05
2,2,9.461429e+05
3,4,2.475643e+06
4,5,1.111857e+06


In [18]:
train.head(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000


### Q1. 업종전환을 한 가게가 있나요?

In [46]:
q1 =train.groupby(['store_id'])['type_of_business'].nunique().unique()
q1

array([1, 0])

In [58]:
q1_2 = train.groupby('store_id')['type_of_business'].nunique()
q1_2[q1_2 > 1] 
# A1: 없다!

Series([], Name: type_of_business, dtype: int64)

### Q2. 상점의 지역 별 분포는 어떻게 되나요?

In [60]:
q2 = train.groupby(['region'])['store_id'].nunique()
q2

region
강원 강릉시      9
강원 삼척시      4
강원 속초시      4
강원 양구군      1
강원 원주시     10
강원 철원군      1
강원 춘천시     10
강원 태백시      2
강원 홍천군      1
강원 횡성군      2
경기 가평군      3
경기 고양시     11
경기 과천시      2
경기 광명시      6
경기 광주시     11
경기 구리시      7
경기 군포시      3
경기 김포시      6
경기 남양주시    16
경기 동두천시     2
경기 부천시     21
경기 성남시     24
경기 수원시     34
경기 시흥시      7
경기 안산시     14
경기 안성시      4
경기 안양시     10
경기 양주시      5
경기 양평군      1
경기 연천군      2
           ..
전남 해남군      2
전북 군산시      9
전북 김제시      1
전북 남원시      1
전북 순창군      1
전북 익산시      6
전북 전주시     20
전북 정읍시      4
제주 서귀포시     9
제주 제주시     22
충남 공주시      5
충남 금산군      1
충남 논산시      2
충남 당진시      5
충남 보령시      4
충남 부여군      1
충남 서산시      5
충남 서천군      1
충남 아산시      8
충남 예산군      1
충남 천안시     21
충남 홍성군      3
충북 단양군      1
충북 옥천군      2
충북 음성군      5
충북 제천시      8
충북 증평군      2
충북 진천군      1
충북 청주시     32
충북 충주시      9
Name: store_id, Length: 180, dtype: int64

### Q3. 상점의 업종 별 분포는 어떻게 되나요?

In [61]:
q3 = train.groupby(['type_of_business'])['store_id'].nunique()
q3

type_of_business
가구 소매업                         3
가전제품 소매업                       3
가정용 세탁업                        5
가정용 직물제품 소매업                   2
간이음식 포장 판매 전문점                18
간판 및 광고물 제조업                   1
건강보조식품 소매업                     2
게임용구, 인형 및 장난감 소매업             2
결혼 상담 및 준비 서비스업                1
경영 컨설팅업                        1
곡물 및 기타 식량작물 재배업               1
골프연습장 운영업                      3
공구 소매업                         1
귀금속 및 관련제품 제조업                 1
그 외 기타 건축자재 도매업                1
그 외 기타 나무제품 제조업                1
그 외 기타 달리 분류되지 않은 개인 서비스업      4
그 외 기타 무점포 소매업                 1
그 외 기타 분류 안된 가정용품 소매업          2
그 외 기타 분류 안된 교육기관              6
그 외 기타 분류 안된 사업지원 서비스업         1
그 외 기타 분류 안된 상품 전문 소매업         7
그 외 기타 분류 안된 오락관련 서비스업         1
그 외 기타 상품 전문 도매업               3
그 외 기타 생활용품 도매업                1
그 외 기타 스포츠 서비스업                2
그 외 기타 스포츠시설 운영업               4
그 외 기타 종합 소매업                  2
그 외 기타 창작 및 예술관련 서비스업          2
기념품, 관광 민예품 및 장식용품 소매업    